<p style="align: center;">
    <img align=center src="../../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

Это домашнее задание будет посвящено полноценному решению задачи машинного обучения.

Есть две части этого домашнего задания:

* Сделать полноценный отчёт о вашей работе: как вы обработали данные, какие модели попробовали и какие результаты получились (максимум 10 баллов). За каждую выполненную часть будет начислено определённое количество баллов.

* Лучшее решение отправить в соревнование на [kaggle](https://www.kaggle.com/t/f50bc21dbe0e42dabe5e32a21f2e5235) (максимум 5 баллов). За прохождение определенных порогов будут начисляться баллы.

Обе части будут проверяться в формате **peer-review**. Т.е. вашу посылку на Stepik будут проверять несколько других студентов и аггрегация их оценок будет выставлена в качестве итоговой оценки. В то же время вам тоже нужно будет проверить несколько других учеников.

Пожалуйста, делайте свою работу чистой и понятной, чтобы облегчить проверку. Если у вас будут проблемы с решением или хочется совета, то пишите в наш чат в телеграме или в лс @runfme. Если вы захотите проаппелировать оценку, то пипшите в лс @runfme.

Во всех пунктах указания - это минимальный набор вещей, которые стоит сделать. Если вы можете сделать какой-то шаг лучше или добавить что-то своё - дерзайте!

## Как проверять?

Ставьте полный балл, если выполнены все рекомендации или сделано что-то более интересное и сложное. За каждый отсустствующий пункт из рекомендаций снижайте оценку на 1 балл.

## Метрика

Перед решением любой задачи важно понимать, как будет оцениваться ваше решение. В данном случае мы используем стандартную для задачи классификации метрику **ROC-AUC**. Ее можно вычислить, используя только предсказанные вероятности и истинные классы без конкретного порога классификации, плюс она работает, даже если классы в данных сильно несбалансированы (примеров одного класса в десятки раз больше примеров другого). Именно поэтому она очень удобна для соревнований.

Посчитать её легко:

In [ ]:
from sklearn.metrics import roc_auc_score

y_true = [
    0,
    1,
    1,
    0,
    1,
]

y_predictions = [
    0.1,
    0.9,
    0.4,
    0.6,
    0.61,
]

roc_auc_score(y_true, y_predictions)

---

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

## Загрузка данных (2 балла)

1. Посмотрите на случайные строчки.

2. Посмотрите, есть ли в датасете незаполненные значения (`nan`) с помощью `data.isna()` или `data.info()` и, если нужно, замените их на что-то. Будет хорошо, если вы построите табличку с количеством `nan` в каждой колонке.

In [ ]:
# загрузим данные
data_train = pd.read_csv('data/train.csv')
data_test = pd.read_csv('data/test.csv')

In [ ]:
# для вашего удобства списки с именами разных столбцов

# числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent',
]

# категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod',
]

feature_cols = num_cols + cat_cols

target_col = 'Churn'

In [ ]:
# помотрим на 5 случайных строк обучающего датасета
data_train.sample(5)

In [ ]:
# помотрим на 5 случайных строк тестового датасета
data_test.sample(5)

In [ ]:
# посмотрим на общую информацию по обучающему датасету
# видим, что незаполненных значений нет,
# однако тип столбца 'TotalSpent' почему-то object, а не float
data_train.info()

In [ ]:
# посмотрим на общую информацию по тестовому датасету
# видим, что незаполненных значений тоже нет,
# а со столбцом 'TotatSpent' та же ситуация
data_test.info()

In [ ]:
# дополнительно проверим значения в каждом из столбцов
for feature in feature_cols:
    print(data_train[feature].value_counts())
    print('-' * 30)
    
print(data_train[target_col].value_counts())

In [ ]:
# видим, что в обучающем датасете в столбце 'TotalSpent'
# есть 9 пробелов, посмотрим на эти записи
data_train[data_train['TotalSpent'] == ' ']

In [ ]:
# видим, что это новые клиенты ('ClientPeriod' == 0),
# поэтому заменим 'TotalSpent' в этих строках на 0.0
data_train.replace(' ', 0.0, inplace=True)

# теперь приведём все значения в колонке 'TotalSpent' к типу float64
data_train['TotalSpent'] = pd.to_numeric(data_train['TotalSpent'])

In [ ]:
# проверим, нет ли такой же ситуации в тестовом датасете
data_test[data_test['TotalSpent'] == ' ']

In [ ]:
# видим два таких же кейса
# проделаем аналогичные манипуляции
data_test.replace(' ', 0.0, inplace=True)
data_test['TotalSpent'] = pd.to_numeric(data_test['TotalSpent'])

In [ ]:
# сохраним
data_train_orig = data_train.copy()
data_test_orig = data_test.copy()

## Анализ данных (3 балла)

1. Для численных призанков постройте гистограмму (`plt.hist`) или boxplot (`plt.boxplot`). Для категориальных посчитайте количество каждого значения для каждого признака. Для каждой колонки надо сделать `data.value_counts()` и построить bar диаграммы (`plt.bar`) или круговые диаграммы (`plt.pie`) (хорошо, если вы сможете это сделать на одном графике с помощью `plt.subplots`). 

2. Посмотрите на распределение целевой переменной и скажите, являются ли классы несбалансированными.

3. (Если будет желание) Поиграйте с разными библиотеками для визуализации - **sns**, **pandas_visual_analysis**, etc.

Второй пункт очень важен, потому что существуют задачи классификации с несбалансированными классами. Например, это может значить, что в датасете намного больше примеров $0$ класса. В таких случаях нужно: 1) не использовать `accuracy` как метрику, 2) использовать методы борьбы с **imbalanced dataset** (обычно, если датасет сильно несбалансирован, т.е. класса $1$ в $20$ раз меньше класса $0$).

In [ ]:
# распределение численных признаков в обучающем датасете
data_train[num_cols].hist(figsize=(15, 8))
plt.show()

In [ ]:
# распределение численных признаков в тестовом датасете
# примерно совпадает с распределением в обучающем датасете
data_test[num_cols].hist(figsize=(15, 8))
plt.show()

In [ ]:
# посмотрим на распределение категориальных
# признаков в обучающем датасете

fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

fig.subplots_adjust(hspace=0.5, wspace=0.05)

for i, feature in enumerate(cat_cols):
    fig.add_subplot(axs[i // 4, i % 4])
    data_train[feature].value_counts().plot(kind='pie', title=feature)
    plt.axis('off')
    
plt.show()

In [ ]:
# посмотрим на распределение категориальных
# признаков в тестовом датасете
# как и в случае численных признаков распределение
# соответствует распределению на обучающем датасете

fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

fig.subplots_adjust(hspace=0.5, wspace=0.05)

for i, feature in enumerate(cat_cols):
    fig.add_subplot(axs[i // 4, i % 4])
    data_test[feature].value_counts().plot(kind='pie', title=feature)
    plt.axis('off')
    
plt.show()

In [ ]:
# распределение целевой переменной
# соотношение классов примерно 3:1
# классы не являются сбалансированными, но и очень
# сильного перекоса в пользу одного из классов тоже нет
data_train[target_col].value_counts().plot(kind='pie', title=target_col)
plt.axis('off')
plt.show()

(Дополнительно) Если вы нашли какие-то ошибки в данных или выбросы, то можете их убрать. Тут можно поэксперементировать с обработкой данных как угодно, но не за баллы.

In [ ]:
# I'm to lazy for this stuff

## Применение линейных моделей (3 балла)

1. Обработайте данные для того, чтобы к ним можно было применить `LogisticRegression`. Т.е. отнормируйте числовые признаки, а категориальные закодируйте с помощью one-hot encoding'а.

2. С помощью кроссвалидации или разделения на train/valid выборку протестируйте разные значения гиперпараметра `C` и выберите лучший (можно тестировать С=100, 10, 1, 0.1, 0.01, 0.001) по метрике ROC-AUC.

Если вы разделяете на train/valid, то используйте `LogisticRegressionCV`. Он сам при вызове `.fit()` подберет параметр `С` (не забудьте передать `scroing='roc_auc'`, чтобы при кроссвалидации сравнивались значения этой метрики, и `refit=True`, чтобы модель обучилась на всём датасете с лучшим параметром `C`). 

(более сложный вариант) Если вы будете использовать кроссвалидацию, то преобразования данных и `LogisticRegression` нужно соединить в один pipeline с помощью `make_pipeline`, как это делалось во втором семинаре. Потом pipeline надо передать в `GridSearchCV`. Для one-hot encoding'a можно испльзовать комбинацию `LabelEncoder` + `OneHotEncoder` (сначала превращаем строчки в числа, а потом числа првращаем в one-hot вектора).

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score

In [ ]:
scaler = StandardScaler()
encoder = OneHotEncoder()

In [ ]:
# выделим численные и категориальные признаки
# а также целевую переменную
num_data_train = data_train[num_cols]
cat_data_train = data_train[cat_cols]
target = data_train[target_col]

In [ ]:
# нормируем и центрируем численные признаки
num_data_train = scaler.fit_transform(num_data_train)
# применяем OHE к категориальным признакам
cat_data_train = encoder.fit_transform(cat_data_train).toarray()

In [ ]:
# сливаем обратно
data_train = np.concatenate((num_data_train, cat_data_train), axis=1)

In [ ]:
# задаём модель и обучаем
logreg_clf = LogisticRegressionCV(scoring='roc_auc', refit=True, random_state=42)

logreg_clf.fit(data_train, target)

Выпишите какое лучшее качество и с какими параметрами вам удалось получить.

In [ ]:
# параметр C
logreg_clf.C_[0]

In [ ]:
# качество на обучающем датасете
target_pred = logreg_clf.predict_proba(data_train)[:, 1]

train_score = roc_auc_score(target, target_pred)
train_score

## Применение градиентного бустинга (2 балла)

Если вы хотите получить баллы за точный ответ, то стоит попробовать градиентный бустинг. Часто градиентный бустинг с дефолтными параметрами даст вам $80\%$ результата за $0\%$ усилий.

Мы будем использовать `CatBoost`, поэтому нам не надо кодировать категориальные признаки. `CatBoost` сделает это сам (в `.fit()` надо передать `cat_features=cat_cols`). А численные признаки нормировать для моделей, основанных на деревьях, не нужно.

1. Разделите выборку на train/valid. Протестируйте `CatBoost` cо стандартными параметрами.

2. Протестируйте разные значения параметров количества деревьев и learning_rate'а и выберите лучшую по метрике ROC-AUC комбинацию. 

(Дополнительно) Есть некоторые сложности с тем, чтобы использовать `CatBoostClassifier` вместе с `GridSearchCV`, поэтому мы не просим использовать кроссвалидацию. Но можете попробовать :)

In [ ]:
import catboost

In [ ]:
# отделим колонку с целевой переменной
data_train = data_train_orig[num_cols + cat_cols]
target = data_train_orig[target_col]

In [ ]:
# разделим обучающий датасет на train/valid
X_train, X_valid, y_train, y_valid = train_test_split(data_train,
                                                      target,
                                                      train_size=0.8,
                                                      random_state=42)

In [ ]:
# запустим со стандартными параметрами
def_catboost_clf = catboost.CatBoostClassifier(cat_features=cat_cols, eval_metric='AUC')

# X_train = data_train
# y_train = target

def_catboost_clf.fit(X_train, y_train)

In [ ]:
# слабенько :(
y_pred = def_catboost_clf.predict_proba(X_valid)[:, 1]

valid_score = roc_auc_score(y_valid, y_pred)
valid_score

In [ ]:
# поиграемся с параметрами
grid_catboost_clf = catboost.CatBoostClassifier(cat_features=cat_cols, eval_metric='AUC')

grid = {
    'n_estimators': [100, 200, 400, 800, 1000],
    'learning_rate': [0.005, 0.01, 0.03, 0.06, 0.1],
}

grid_search_result = grid_catboost_clf.grid_search(grid,
                                                   X=data_train,
                                                   y=target,
                                                   plot=True)

In [ ]:
# видим, что лучший результат получился на параметрах
# n_estimators = 200
# learning_rate = 0.1
# обучим классификатор с такими параметрами
best_catboost_clf = catboost.CatBoostClassifier(cat_features=cat_cols,
                                                eval_metric='AUC',
                                                n_estimators=200,
                                                learning_rate=0.1)

# X_train = data_train
# y_train = target

best_catboost_clf.fit(X_train, y_train)

Выпишите какое лучшее качество и с какими параметрами вам удалось получить.

In [ ]:
grid_search_result['params']

In [ ]:
y_pred = best_catboost_clf.predict_proba(X_valid)[:, 1]

valid_score = roc_auc_score(y_valid, y_pred)
valid_score

## Предсказания

In [ ]:
# best_model = logreg_clf

# scaler = StandardScaler()
# encoder = OneHotEncoder()

# num_data_test = data_test[num_cols]
# cat_data_test = data_test[cat_cols]

# num_data_test = scaler.fit_transform(num_data_test)
# cat_data_test = encoder.fit_transform(cat_data_test).toarray()

# data_test = np.concatenate((num_data_test, cat_data_test), axis=1)

# data_submission = pd.read_csv('data/submission.csv')

# data_submission['Churn'] = best_model.predict_proba(data_test)[:, 1]
# data_submission.to_csv('data/my_submission.csv', index=False)

In [ ]:
# best_model = def_catboost_clf

# data_submission = pd.read_csv('data/submission.csv')

# data_submission['Churn'] = best_model.predict_proba(data_test_orig)[:, 1]
# data_submission.to_csv('data/my_submission.csv', index=False)

In [ ]:
best_model = best_catboost_clf

data_submission = pd.read_csv('data/submission.csv')

data_submission['Churn'] = best_model.predict_proba(data_test_orig)[:, 1]
data_submission.to_csv('data/my_submission.csv', index=False)

## Kaggle (5 баллов)

Как выставить баллы:

1. 1 >= roc auc > 0.84 - это 5 баллов

2. 0.84 >= roc auc > 0.7 - это 3 балла

3. 0.7 >= roc auc > 0.6 - это 1 балл

4. 0.6 >= roc auc - это 0 баллов

Для выполнения задания необходимо выполнить следующие шаги:

* Зарегистрироваться на платформе [kaggle.com](https://www.kaggle.com/). Процесс выставления оценок будет проходить при подведении итогового рейтинга. Пожалуйста, укажите во вкладке Team -> Team name свои имя и фамилию в формате Имя_Фамилия (важно, чтобы имя и фамилия совпадали с данными на Stepik).

* Обучить модель, получить файл с ответами в формате `.csv` и сдать его в конкурс. Пробуйте и экспериментируйте. Обратите внимание, что вы можете выполнять до $20$ попыток сдачи на kaggle в день.

* После окончания соревнования отправить итоговый ноутбук с решением на степик.

* После дедлайна проверьте посылки других участников по критериям. Для этого надо зайти на степик, скачать их ноутбук и проверить скор в соревновании.

---

**Для проверяющих:**

* stepik: https://stepik.org/users/62768875

* telegram: @nalysann